In [ ]:
%load_ext autoreload
%autoreload 2

from context import uncertify

In [ ]:
import logging
from uncertify.log import setup_logging
setup_logging()
LOG = logging.getLogger(__name__)

# Matplotlib DEBUG logging spits out a whole bunch of crap
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)

In [ ]:
from pathlib import Path

import torch
import torchvision
from torchvision.transforms.transforms import Compose
from tqdm import tqdm
try:
    tqdm._instances.clear()
except:
    pass
import seaborn as sns
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
plt.rc('font', family='serif')
matplotlib.rcParams.update({'font.size': 18})

from uncertify.data.np_transforms import NumpyReshapeTransform, Numpy2PILTransform
from uncertify.data.datasets import GaussianNoiseDataset

from uncertify.data.dataloaders import dataloader_factory, DatasetType

from uncertify.io.models import load_ensemble_models
from uncertify.evaluation.ood_experiments import run_ood_evaluations, run_ood_to_ood_dict
from uncertify.evaluation.configs import OODDetectionResults
from uncertify.evaluation.inference import yield_inference_batches
from uncertify.evaluation.statistics import aggregate_slice_wise_statistics

from uncertify.visualization.histograms import plot_multi_histogram
from uncertify.visualization.ood_scores import plot_ood_scores, plot_most_least_ood, plot_samples_close_to_score
from uncertify.visualization.grid import imshow_grid

from uncertify.common import DATA_DIR_PATH

In [ ]:
# Define some paths and high level parameters
RUN_DIR_PATH = Path('/media/juniors/2TB_internal_HD/lightning_logs/train_vae/')
RUN_VERSIONS = range(2)
CHECKPOINT_PATHS = [RUN_DIR_PATH / f'version_{version}/checkpoints/last.ckpt' for version in RUN_VERSIONS]

In [ ]:
ensemble_models = load_ensemble_models(DATA_DIR_PATH / 'ensemble_models', [f'model{idx}.ckpt' for idx in RUN_VERSIONS])

In [ ]:
batch_size = 8
USE_N_BATCHES = 3

PROCESSED_DIR_PATH = Path('/media/juniors/2TB_internal_HD/datasets/processed/')

brats_t2_path    = PROCESSED_DIR_PATH / 'brats17_t2_bc_std_bv3.5_l10.hdf5'
brats_t2_hm_path = PROCESSED_DIR_PATH / 'brats17_t2_hm_bc_std_bv-3.5.hdf5'
brats_t1_path    = PROCESSED_DIR_PATH / 'brats17_t1_bc_std_bv3.5_l10.hdf5'
brats_t1_hm_path = PROCESSED_DIR_PATH / 'brats17_t1_hm_bc_std_bv-3.5.hdf5'
camcan_t2_val_path   = DATA_DIR_PATH  / 'processed/camcan_val_t2_hm_std_bv3.5_xe.hdf5'
camcan_t2_train_path = DATA_DIR_PATH  / 'processed/camcan_train_t2_hm_std_bv3.5_xe.hdf5'

_, brats_val_t2_dataloader    = dataloader_factory(DatasetType.BRATS17, batch_size=batch_size, val_set_path=brats_t2_path, shuffle_val=True, num_workers=12)
_, brats_val_t1_dataloader    = dataloader_factory(DatasetType.BRATS17, batch_size=batch_size, val_set_path=brats_t1_path, shuffle_val=False, num_workers=12)
_, brats_val_t2_hm_dataloader = dataloader_factory(DatasetType.BRATS17, batch_size=batch_size, val_set_path=brats_t2_hm_path, shuffle_val=False, num_workers=12)
_, brats_val_t1_hm_dataloader = dataloader_factory(DatasetType.BRATS17, batch_size=batch_size, val_set_path=brats_t1_hm_path, shuffle_val=False, num_workers=12)

hflip_transform = torchvision.transforms.Compose([
    NumpyReshapeTransform((200, 200)),
    Numpy2PILTransform(),
    torchvision.transforms.Resize((128, 128)),
    torchvision.transforms.RandomHorizontalFlip(p=1.0),
    torchvision.transforms.ToTensor()
])
vflip_transform = torchvision.transforms.Compose([
    NumpyReshapeTransform((200, 200)),
    Numpy2PILTransform(),
    torchvision.transforms.Resize((128, 128)),
    torchvision.transforms.RandomVerticalFlip(p=1.0),
    torchvision.transforms.ToTensor()
])

_, brats_val_t2_hflip_dataloader = dataloader_factory(DatasetType.BRATS17, batch_size=batch_size, val_set_path=brats_t2_path, shuffle_val=False, num_workers=12, transform=hflip_transform)
_, brats_val_t2_vflip_dataloader = dataloader_factory(DatasetType.BRATS17, batch_size=batch_size, val_set_path=brats_t2_path, shuffle_val=False, num_workers=12, transform=vflip_transform)

camcan_train_dataloader, camcan_val_dataloader = dataloader_factory(DatasetType.CAMCAN, batch_size=batch_size, val_set_path=camcan_t2_val_path, train_set_path=camcan_t2_train_path, shuffle_val=False, shuffle_train=True, num_workers=12)

noise_set = GaussianNoiseDataset()
noise_loader = DataLoader(noise_set, batch_size=batch_size)

_, mnist_val_dataloader = dataloader_factory(DatasetType.MNIST, batch_size=batch_size, transform=torchvision.transforms.Compose([
                                                                        torchvision.transforms.Resize((128, 128)),
                                                                        torchvision.transforms.ToTensor()
                                                                    ])
                         )

dataloader_dict = {'BraTS T2 val': brats_val_t2_dataloader, 
                   #'BraTS T1 val': brats_val_t1_dataloader, 
                   #'BraTS T2 HM val': brats_val_t2_hm_dataloader, 
                   #'BraTS T1 HM val': brats_val_t1_hm_dataloader,
                   #'CamCAN train': camcan_train_dataloader,
                   #'Gaussian noise': noise_loader,
                   #'MNIST': mnist_val_dataloader,
                   #'BraTS T2 HFlip': brats_val_t2_hflip_dataloader,
                   #'BraTS T2 VFlip': brats_val_t2_vflip_dataloader
}
brats_dataloader_dict = {key: val for key, val in dataloader_dict.items() if 'BraTS' in key}

for name, dataloader in dataloader_dict.items(): 
    print(f'{name:15} dataloader: {len(dataloader)} batches (batch_size: {dataloader.batch_size}) -> {len(dataloader) * dataloader.batch_size} samples.')

# OOD Detection Evaluation for different OOD datasets

In [ ]:
for batch in yield_inference_batches(brats_val_t2_dataloader, ensemble_models[0], 1, 1.3):
    b = batch

In [ ]:
from uncertify.visualization.grid import imshow_grid
import torchvision

In [ ]:
orig_grid = torchvision.utils.make_grid(b.scan, padding=0)
masked_grid = torchvision.utils.make_grid(b.scan * b.mask, padding=0)

In [ ]:
imshow_grid(orig_grid, one_channel=True, axis='off', vmin=-3.5, figsize=(12, 12))
imshow_grid(masked_grid, one_channel=True, axis='off', vmin=-3.5, figsize=(12, 12))

In [ ]:
b

In [ ]:
torch.sum(batch.mask, dim=(1, 2, 3))

In [ ]:
a = torch.tensor([6, 8])

In [ ]:
b = torch.tensor([2, 4])

In [ ]:
a / b

In [ ]:
run_ood_evaluations(camcan_train_dataloader, dataloader_dict, ensemble_models, residual_threshold=1.35, max_n_batches=3)

# WAIC Score Histograms

In [ ]:
NUM_BACTHES = 3

dataloader_dict = {'BraTS T2 val': brats_val_t2_dataloader, 
                   #'BraTS T1 val': brats_val_t1_dataloader, 
                   'BraTS T2 HM val': brats_val_t2_hm_dataloader, 
                   #'BraTS T1 HM val': brats_val_t1_hm_dataloader,
                   'CamCAN train': camcan_train_dataloader,
                   #'Gaussian noise': noise_loader,
                   #'MNIST': mnist_val_dataloader
}

waic_dict = run_ood_to_ood_dict(dataloader_dict, ensemble_models, num_batches=NUM_BACTHES)

In [ ]:
plot_ood_scores(waic_dict, score_label='WAIC', dataset_name_filters=['BraTS T2 val'], modes_to_include=['healthy'])

In [ ]:
plot_most_least_ood(waic_dict, 'BraTS T2 val')
plot_most_least_ood(waic_dict, 'BraTS T2 HM val')
plot_most_least_ood(waic_dict, 'CamCAN train', do_lesional=False)

In [ ]:
plot_samples_close_to_score(waic_dict, 'BraTS T2 val', min_score=0.95, max_score=1.2)
plot_samples_close_to_score(waic_dict, 'BraTS T2 HM val', min_score=0.85, max_score=1.2)
plot_samples_close_to_score(waic_dict, 'CamCAN train', do_lesional=False, min_score=0.8, max_score=0.9)

# Slice-Wise Statistics

In [ ]:
import pandas as pd
import seaborn as sns
import scipy

In [ ]:
STATISTICS = ['kl_div', 'rec_err', 'elbo']

statistics_dict = aggregate_slice_wise_statistics(ensemble_models[0], camcan_train_dataloader, STATISTICS, max_n_batches=100)

In [ ]:
df = pd.DataFrame(statistics_dict)
df = df.loc[np.invert(df.is_empty)]

In [ ]:
g = sns.pairplot(df, vars=['kl_div', 'rec_err', 'elbo'])
g.map_upper(sns.scatterplot)
g.map_lower(sns.kdeplot, shade=True, thresh=0.05, alpha=0.7)
g.map_diag(sns.kdeplot)

In [ ]:
g = sns.pairplot(df, vars=['kl_div', 'rec_err', 'elbo'], hue='is_lesional')
g.map_upper(sns.scatterplot)
g.map_lower(sns.kdeplot, shade=True, thresh=0.05, alpha=0.7)
g.map_diag(sns.kdeplot)

In [ ]:
kde_kv = scipy.stats.gaussian_kde(df['kl_div'])
kde_rec = scipy.stats.gaussian_kde(df['rec_err'])

In [ ]:
type(kde_kv)

In [ ]:
kde_x = np.linspace(0, 0.2, 100)
rec_x = np.linspace(0, 2.0, 100)
plt.figure()
plt.plot(kde_x, [kde_kv(x_val) for x_val in kde_x])
plt.figure()
plt.plot(rec_x, [kde_rec(x_val) for x_val in rec_x])

In [ ]:
from uncertify.evaluation.dose import compute_slice_wise_dose_kde_scores, compute_slice_wise_dose_scores
from uncertify.evaluation.statistics import fit_statistics, aggregate_slice_wise_statistics

In [ ]:
DOSE_STATISTICS = ['rec_err', 'kl_div', 'elbo']
MAX_N_BATCHES = 10

statistics_dict = aggregate_slice_wise_statistics(ensemble_models[0], camcan_train_dataloader, DOSE_STATISTICS, max_n_batches=MAX_N_BATCHES)
kde_func_dict = fit_statistics(statistics_dict)
dose_kde_dict = compute_slice_wise_dose_kde_scores(ensemble_models[0], brats_val_t2_dataloader, kde_func_dict, DOSE_STATISTICS, MAX_N_BATCHES)
dose_scores = compute_slice_wise_dose_scores(dose_kde_dict)

In [ ]:
dose_df = pd.DataFrame(dose_kde_dict)

In [ ]:
g = sns.pairplot(dose_df, vars=['kl_div', 'rec_err', 'elbo'], hue='is_lesional')
g.map_upper(sns.scatterplot)
g.map_lower(sns.kdeplot, shade=True, thresh=0.05, alpha=0.7)
g.map_diag(sns.kdeplot)